In [3]:
! pip install streamlit -q



     |████████████████████████████████| 9.2 MB 5.7 MB/s 
     |████████████████████████████████| 164 kB 46.6 MB/s 
     |████████████████████████████████| 111 kB 51.3 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 4.3 MB 44.6 MB/s 
     |████████████████████████████████| 180 kB 62.8 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 126 kB 38.4 MB/s 
     |████████████████████████████████| 791 kB 50.4 MB/s 
     |████████████████████████████████| 374 kB 50.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.24 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.6.0 which is incompatible.
google-colab 1.0.0 requires ipytho

In [4]:
%%writefile option_pricer.py

import pandas as pd
import streamlit as st
import time 
import numpy as np
import requests

from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date, timedelta
from pandas import DataFrame



st.write("""
         
# Option Pricer

Enter the required fields

""")

def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

asset = user_input = st.text_input("Asset")




def get_spot(asset):  
  
  url = "https://alpha-vantage.p.rapidapi.com/query"

  querystring = {"from_currency":asset,"function":"CURRENCY_EXCHANGE_RATE","to_currency":"USD"}

  headers = {
        'x-rapidapi-host': "alpha-vantage.p.rapidapi.com",
        'x-rapidapi-key': "296091f1a0msh573dbf3b0f29838p1b1343jsn448791d5004f"
        }

  response = requests.request("GET", url, headers=headers, params=querystring)

  df = pd.DataFrame(response.json()).T

  return pd.to_numeric(df.iloc[0, 4])

if asset:
  try:
    price = get_spot(asset) 
  except:
    print('Error getting spot price please enter manually')
    price = float(st.number_input("Spot Price"))

  st.write('Spot price of {} is {}'.format(asset, price))

  call_strikes = [price*1.05, price*1.1, price*1.15, price*1.2, price*1.25]

  put_strikes = [price*.95, price*0.9, price*0.85, price*0.8, price*0.75]

  days_to_expiry = st.number_input("Days to Expiry", value=7)

  hour_of_expiry_utc = st.number_input("UTC Hour of Expiry", value=15)

  forward_yield = st.number_input("Forward Yield - e.g. for 5% input 5", 5)/100

  bid_vol = st.number_input("Bid IV - e.g. if 80% input 80", value=70)/100

  offer_vol = st.number_input("Offer IV - e.g. if 80% input 80", value=70)/100

  spread = st.number_input("Spread each side", value=0.03)

  now = datetime.now()

  expiry = now+timedelta(days=days_to_expiry)

  expiry = expiry.replace(hour=hour_of_expiry_utc, minute=00, second=00)

  fraction_of_year = (expiry-now).total_seconds()/60/60/24/365

  forward_price = price*fraction_of_year*forward_yield

  custom_expiry = st.date_input("Custom Expiry Date - N.B. Do NOT need both this and Expiry days, BUT you do need to make sure the UTC hour of expiry is correct", (now-timedelta(days=1)))

  custom_strike = st.number_input("Custom Strike")

  if st.button('Click when all fields correct.  Program will compute 95%-75% Puts and 105%-125% Calls'):

    if custom_expiry > now.date():
      expiry = datetime(custom_expiry.year, custom_expiry.month, custom_expiry.day, hour_of_expiry_utc)
      fraction_of_year = (expiry-now).total_seconds()/60/60/24/365

    if custom_strike:
      bid = round(bs_call(S=price, K=custom_strike, T=fraction_of_year, r=forward_yield, sigma=bid_vol-spread), 2)
      offer = round(bs_call(S=price, K=custom_strike, T=fraction_of_year, r=forward_yield, sigma=offer_vol+spread), 2)

      st.write('Expiry  = {}. {} Spot = **{}** -----    **{}** strike calls         **${}/${}**,      vols = **{}%/{}%**'.format(expiry, asset, price,  custom_strike, bid, offer, (bid_vol-spread)*100, (offer_vol+spread)*100))
      st.write('')
      bid = round(bs_put(S=price, K=custom_strike, T=fraction_of_year, r=forward_yield, sigma=bid_vol-spread), 2)
      offer = round(bs_put(S=price, K=custom_strike, T=fraction_of_year, r=forward_yield, sigma=offer_vol+spread), 2)

      st.write('Expiry  = {}. {} Spot = **{}** -----    **{}** strike puts          **${}/${}**,      vols = **{}%/{}%**'.format(expiry, asset, price,  custom_strike, bid, offer, (bid_vol-spread)*100, (offer_vol+spread)*100))
      st.write('')

    else:
      for c in call_strikes:
        c = round(c, -1)
        bid = round(bs_call(S=price, K=c, T=fraction_of_year, r=forward_yield, sigma=bid_vol-spread), 2)
        offer = round(bs_call(S=price, K=c, T=fraction_of_year, r=forward_yield, sigma=offer_vol+spread), 2)

        st.write('Expiry  = {}. {} Spot = **{}** -----    **{}** strike calls         **${}/${}**,      vols = **{}%/{}%**'.format(expiry, asset, price,  c, bid, offer, (bid_vol-spread)*100, (offer_vol+spread)*100))
        st.write('')
      for c in put_strikes:
        c = round(c, -1)
        bid = round(bs_put(S=price, K=c, T=fraction_of_year, r=forward_yield, sigma=bid_vol-spread), 2)
        offer = round(bs_put(S=price, K=c, T=fraction_of_year, r=forward_yield, sigma=offer_vol+spread), 2)

        st.write('Expiry  = {}. {} Spot = **{}** -----    **{}** strike puts          **${}/${}**,      vols = **{}%/{}%**'.format(expiry, asset, price,  c, bid, offer, (bid_vol-spread)*100, (offer_vol+spread)*100))
        st.write('')



Overwriting option_pricer.py


In [ ]:
!streamlit run option_pricer.py & npx localtunnel --port 8501

npx: installed 22 in 2.99s
your url is: https://selfish-lionfish-57.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.72.47.169:8501

Error getting spot price please enter manually
Error getting spot price please enter manually
2022-01-03 09:36:35.278 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/script_runner.py", line 354, in _run_script
    exec(code, module.__dict__)
  File "/content/option_pricer.py", line 102, in <module>
    bid = round(bs_call(S=price, K=custom_strike, T=fraction_of_year, r=forward_yield, sigma=bid_vol-spread), 2)
  File "/content/option_pricer.py", line 29, in bs_call
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  File "/content/option_pricer.py", line 24, in d1
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
ValueError: math domain error

Error getting spot price please enter manually
Error get